# Imports

In [1]:
import pandas as pd; import seaborn as sb; import warnings; import scipy; import re; 
import os; from analysis_helpers import *; import itertools; from scipy import stats
import random; import pandas as pd; import numpy as np; from sklearn import datasets, linear_model; 
from sklearn.linear_model import LinearRegression; import statsmodels.api as sm
from scipy import stats; from itertools import groupby; from operator import itemgetter;
import csv; import time
warnings.filterwarnings('ignore')
%matplotlib inline

# Import behavioral data 

Here, we ran a single presentation and memory run and recorded and time stamped the filename of each presented stimulus, its location, millisecond timings on window flips, etc, to verify the precision and accuracy of the data in our behavioral data files

In [2]:
with open('/Users/kirstenziman/Desktop/FIIIIINAL_2019_Sep_05/test_data/variable_attn_check_FINAL.log') as file:
    lines = file.read().splitlines()
    lines = [[lines[x]] for x in range(0, len(lines))]
    lines.insert(0,["DATA"])

    with open('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/variable_attn_check_FINAL.csv', 'w+') as csvfile:
        w = csv.writer(csvfile)
        w.writerows(lines)

# Log file data

In [3]:
log_file = pd.read_csv('/Users/kirstenziman/Desktop/FINAL_EYE_2019_Sep_05/test_data/variable_attn_check_FINAL.csv')
log_file = log_file.reset_index()
log_file['TIME'],log_file['WARNING'],log_file['MESSAGE'] = log_file['DATA'].str.split('\t', 4).str
log_file = log_file.fillna('NAN VALUE')

In [4]:
log_file.head()

,index,DATA,TIME,WARNING,MESSAGE
0,0,2.6971 \tWARNING \twarning,2.6971,WARNING,warning
1,1,2.6971 \tEXP \texperiment,2.6971,EXP,experiment
2,2,2.6971 \tDATA \tdata,2.6971,DATA,data
3,3,2.6972 \tINFO \tinfo,2.6972,INFO,info
4,4,2.6972 \tINFO \tcurrent time: 1567723717.78,2.6972,INFO,current time: 1567723717.78


In [5]:
log_file['TIME'] = log_file['TIME'].str.replace(' ','')
log_file['TIME'] = log_file['TIME'].astype(float, errors='ignore')

In [6]:
p = log_file[(log_file['DATA'].str.contains('COMPOSITE'))
    & ((log_file['DATA'].str.contains('ON')))].melt(id_vars=['MESSAGE'], value_vars=['TIME'])

In [7]:
p['utc'] = [x[-13:] for x in list(p['MESSAGE'])]

In [8]:
p.head()

,MESSAGE,variable,value,utc
0,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/pr...,TIME,9.4308,1567723731.14
1,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/pr...,TIME,9.4475,1567723731.15
2,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/pr...,TIME,16.0661,1567723737.77
3,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/pr...,TIME,16.0830,1567723737.79
4,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/pr...,TIME,22.6846,1567723744.4


In [9]:
uncued = p[ (p['MESSAGE'].str.contains('COMPOSITE 2')) & (p['MESSAGE'].str.contains('ON')) & (~p['MESSAGE'].str.contains('practice'))]
cued   = p[ (p['MESSAGE'].str.contains('COMPOSITE 1')) & (p['MESSAGE'].str.contains('ON')) & (~p['MESSAGE'].str.contains('practice'))]
cued = cued.rename(columns={"MESSAGE": "cued_onset"})
uncued = uncued.rename(columns={"MESSAGE": "uncued_onset"})

cued   = cued.reset_index(drop=True)
uncued = uncued.reset_index(drop=True)

In [10]:
cued

,cued_onset,variable,value,utc
0,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,66.9658,1567723788.66
1,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,74.4192,1567723796.12
2,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,82.3203,1567723804.03
3,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,89.9239,1567723811.63
4,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,97.2761,1567723818.98
5,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,104.7948,1567723826.5
6,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,112.5473,1567723834.25
7,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,120.3997,1567723842.1
8,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,128.0688,1567723849.78
9,WIN FLIP : STIMULUS COMPOSITE 1: ../../stim/co...,TIME,135.3052,1567723857.01


In [11]:
uncued

,uncued_onset,variable,value,utc
0,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,66.9825,1567723788.69
1,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,74.4360,1567723796.14
2,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,82.3376,1567723804.04
3,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,89.9400,1567723811.64
4,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,97.2922,1567723819.0
5,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,104.8118,1567723826.52
6,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,112.5631,1567723834.27
7,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,120.4166,1567723842.12
8,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,128.0860,1567723849.79
9,WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/co...,TIME,135.3213,1567723857.03


In [12]:
uncued['utc'] = uncued['utc'].astype(float)
cued['utc']   = cued['utc'].astype(float)

In [13]:
uncued['uncued_onset'][0]

'WIN FLIP : STIMULUS COMPOSITE 2: ../../stim/composite/00779941205fa_sunagdzlvkqtpnfxfcg.jpg ON 1567723788.69'

# average difference in onset times between Cued and Uncued Composite in a single test block (~1 frame)

In [14]:
x = list(uncued['utc']-cued['utc'])

In [15]:
print('Average number of seconds cued composite shows before uncued composite in a test run '+str(np.mean(x)))

Average number of seconds cued composite shows before uncued composite in a test run 0.018000006675720215


# Check A: order of stimuli, log versus csv files

## verify stimulus orders same as in behavioral csv

### load behavioral csv

In [16]:
sub_list = []
data_dir = '/Users/kirstenziman/Desktop/FIIIIINAL_2019_Sep_05'

for sub_dir in os.listdir(data_dir):
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))
    print(sub_dir)
    
exp_raw = pd.concat(sub_list)
#exp1_raw.to_csv('../parsed_data/behavioral_data_sustained.csv')

test_data


### Cued composite images are in the same order in log file and csv

In [17]:
# cued composite stimuli from log file
[x[53:91] for x in list(cued['cued_onset'])]

['/00952960627fa_sunathzzbcwjykldrem.jpg',
 '/00650941031fa_sunadikxpvqeabxrvvc.jpg',
 '/00498960627fa_sunabzjxlmkwnwzimua.jpg',
 '/00676941121fa_sunadpnqovzckyikrjo.jpg',
 '/00919960620fa_sunapmfmzadmosuogaz.jpg',
 '/00276940422fa_sunabmytpyhsgigllxu.jpg',
 '/00848940307fa_sunaivdylrawbmhryfp.jpg',
 '/00926960627Fa_sunapqwoeotxshkukbh.jpg',
 '/00163931230fb_sunaavodbcpygkggnfp.jpg',
 '/00794960620fa_sunagtqhhwxjeuxdebe.jpg']

In [18]:
# cued composite stimuli from csv
exp_raw[exp_raw['Trial Type']=='Presentation']['Cued Composite']

0    00952960627fa_sunathzzbcwjykldrem.jpg
1    00650941031fa_sunadikxpvqeabxrvvc.jpg
2    00498960627fa_sunabzjxlmkwnwzimua.jpg
3    00676941121fa_sunadpnqovzckyikrjo.jpg
4    00919960620fa_sunapmfmzadmosuogaz.jpg
5    00276940422fa_sunabmytpyhsgigllxu.jpg
6    00848940307fa_sunaivdylrawbmhryfp.jpg
7    00926960627Fa_sunapqwoeotxshkukbh.jpg
8    00163931230fb_sunaavodbcpygkggnfp.jpg
9    00794960620fa_sunagtqhhwxjeuxdebe.jpg
Name: Cued Composite, dtype: object

### Uncued composite images are in the same order in log file and csv

In [19]:
# uncued composite from log
[x[53:91] for x in list(uncued['uncued_onset'])]

['/00779941205fa_sunagdzlvkqtpnfxfcg.jpg',
 '/00838940307fa_sunaiozxhxgtfrmoagh.jpg',
 '/00905960530fa_sunanmoqeyprjjjfxjt.jpg',
 '/00909960530_sunaobtizabegtwfwul.jpg O',
 '/00543940519fa_sunacepzfrmtkiqonpy.jpg',
 '/00142931230fb_sunaapxkpvrhpjplsok.jpg',
 '/00681941121fa_sunadqmjrhfjyuzyvdd.jpg',
 '/00836940307fa_sunainqelldeebvxtcz.jpg',
 '/00270940422fb_sunabllobxadqcyttej.jpg',
 '/00752960620fa_sunafbwnojpqeyzssoe.jpg']

In [20]:
# uncued composite from csv
exp_raw[exp_raw['Trial Type']=='Presentation']['Uncued Composite']

0    00779941205fa_sunagdzlvkqtpnfxfcg.jpg
1    00838940307fa_sunaiozxhxgtfrmoagh.jpg
2    00905960530fa_sunanmoqeyprjjjfxjt.jpg
3      00909960530_sunaobtizabegtwfwul.jpg
4    00543940519fa_sunacepzfrmtkiqonpy.jpg
5    00142931230fb_sunaapxkpvrhpjplsok.jpg
6    00681941121fa_sunadqmjrhfjyuzyvdd.jpg
7    00836940307fa_sunainqelldeebvxtcz.jpg
8    00270940422fb_sunabllobxadqcyttej.jpg
9    00752960620fa_sunafbwnojpqeyzssoe.jpg
Name: Uncued Composite, dtype: object

### Onset times differ by fractions of a second

In [21]:
d = {}

# aggregate onset times (two onsets in log file, one for cued composite and one for uncued; csv has only one onset time assigned to both)
d['csv_cued_onset'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus Onset'])
d['log_cued_onset'] = list(cued['utc'])
d['log_uncued_onset'] = list(uncued['utc'])

# aggregate stim end times
p = log_file[(log_file['DATA'].str.contains('OFF')) & ((log_file['DATA'].str.contains('COMPOSITE'))) & ((~log_file['DATA'].str.contains('practice')))]
p['utc'] = [x[-13:] for x in list(p['DATA'])]
d['log_composite_off'] = [float(x) for x in p['utc']]
d['csv_composite_off'] = list(exp_raw[exp_raw['Trial Type']=='Presentation']['Stimulus End'])

### check time diffs

In [22]:
d = pd.DataFrame(d)
d

,csv_cued_onset,log_cued_onset,log_uncued_onset,log_composite_off,csv_composite_off
0,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
1,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
2,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
3,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
4,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
5,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
6,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
7,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
8,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09
9,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09,1.567724e+09


In [23]:
d['cue_on_diff'] = d['csv_cued_onset']-d['log_cued_onset']
d['uncue_on_diff'] = d['csv_cued_onset']-d['log_uncued_onset']

print('mean onset difference between csv and log file, cued composites: '+str(d['cue_on_diff'].mean()))
print('^^ this is approximately two frames')
print()

print('mean onset difference between csv andn log file, uncued composites: '+str(d['uncue_on_diff'].mean()))
print('^^ this is approximately one frame')

mean onset difference between csv and log file, cued composites: 0.03289008140563965
^^ this is approximately two frames

mean onset difference between csv andn log file, uncued composites: 0.014890074729919434
^^ this is approximately one frame


In [24]:
d['composite_off_diff'] = d['csv_composite_off']-d['log_composite_off']

print('mean end time difference between csv and log file, for both composites: '+str(d['composite_off_diff'].mean()))
print('^^ this is approximately one frame')

mean end time difference between csv and log file, for both composites: -0.01705060005187988
^^ this is approximately one frame


# Check Memory Images

Pull ordered memory images from log file and csv

In [25]:
image_dict = {}
image_dict['mem_from_log'] = [x[-45:-20] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
image_dict['mem_from_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Memory Image'])

In [26]:
pd.DataFrame(image_dict)

,mem_from_log,mem_from_csv
0,tim/single/00952960627fa.,00952960627fa.jpg
1,ngle/sunapqwoeotxshkukbh.,sunapqwoeotxshkukbh.jpg
2,ngle/sunafvwfsmqkjrutjta.,sunafvwfsmqkjrutjta.jpg
3,ngle/sunabllobxadqcyttej.,sunabllobxadqcyttej.jpg
4,ngle/sunacepzfrmtkiqonpy.,sunacepzfrmtkiqonpy.jpg
5,ngle/sunaiycurtqsfkvcaso.,sunaiycurtqsfkvcaso.jpg
6,tim/single/00543940519fa.,00543940519fa.jpg
7,tim/single/00775941205fa.,00775941205fa.jpg
8,tim/single/00779941205fa.,00779941205fa.jpg
9,stim/single/00926960627Fa,00926960627Fa.jpg


Onset times from log

In [27]:
d2={}

In [28]:
onset_list = [x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('ON')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_onset_log'] = [float(x.replace("c", "")) for x in onset_list]
#onset_list

Onset times from csv

In [29]:
d2['memory_onset_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus Onset'])

Aggregate end times from log and csv

In [30]:
d2['memory_off_log']=[x[-13:] for x in list(log_file[(log_file['DATA'].str.contains('MEMORY')) & (log_file['DATA'].str.contains('OFF')) & (~log_file['DATA'].str.contains('practice'))]['DATA'])]
d2['memory_off_csv'] = list(exp_raw[exp_raw['Trial Type']=='Memory']['Stimulus End'])

In [31]:
d2 = pd.DataFrame(d2)
d2.head()

,memory_onset_log,memory_onset_csv,memory_off_log,memory_off_csv
0,1.567724e+09,1.567724e+09,1567723866.32,1.567724e+09
1,1.567724e+09,1.567724e+09,1567723869.57,1.567724e+09
2,1.567724e+09,1.567724e+09,1567723872.82,1.567724e+09
3,1.567724e+09,1.567724e+09,1567723876.08,1.567724e+09
4,1.567724e+09,1.567724e+09,1567723879.31,1.567724e+09


In [32]:
d2['mem_onset_diff'] = d2['memory_onset_csv'] - d2['memory_onset_log']

In [33]:
print('Average difference in memory image onset time between csv and log: '+str(d2['mem_onset_diff'].mean()))

Average difference in memory image onset time between csv and log: 0.031040257215499877


In [34]:
d2['memory_off_log'] = pd.to_numeric(d2['memory_off_log'])
d2['mem_off_diff'] = d2['memory_off_csv'] - d2['memory_off_log']

In [35]:
print('Average difference in end time between csv and log: '+str(d2['mem_off_diff'].mean()))

Average difference in end time between csv and log: -0.12692257165908813


this longer lag in end time is likely due to the amount of time taken to extract the rating from the rating scale. Can time this, to check, and also move the rating scale data pulling code to after the stimulus has been pulled from the screen

# Check Responses and Reaction Times

Here we see that the ratings and reaction times are the same across the log files and csv. Note that the log file does not record a spearate line for the rating scale history when no response is recorded. So, every Nan in the memory data from the csv corresponds with a skipped row in the list below.

In [36]:
# Note, we do not show the first four rating scales because these are from the practice/instruction round
# that is, we ignore the first 8 list items (RT and response info (2) for each practice image (4) --> 2*4 = 8)

print(len([x for x in list(log_file[((log_file['DATA'].str.contains('history')) | (log_file['DATA'].str.contains('RT')))]['DATA'])][8:]))
print()

[x for x in list(log_file[((log_file['DATA'].str.contains('history')) | (log_file['DATA'].str.contains('RT')))]['DATA'])][8:]

72



['142.9275 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.520',
 '142.9275 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (1, 0.52)]',
 '149.9966 \tDATA \tRatingScale unnamed RatingScale: rating RT=1.020',
 '149.9966 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (4, 1.02)]',
 '152.9806 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.753',
 '152.9807 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (2, 0.753)]',
 '155.9822 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.520',
 '155.9822 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (2, 0.519)]',
 '159.5002 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.787',
 '159.5002 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (1, 0.787)]',
 '162.3167 \tDATA \tRatingScale unnamed RatingScale: rating RT=0.353',
 '162.3167 \tDATA \tRatingScale unnamed RatingScale: history=[(None, 0.0), (3, 0.353)]',
 '165.4024 \tDATA \tRatingScale unnamed Ra

In [37]:
### show ratings and reaction times from csv file

# Note that above we see there are 4 ratings that do not appear 
# (participant did not rate the image) and we have four Nan values below

In [38]:
exp_raw[exp_raw['Trial Type']=='Memory'][['Familiarity Rating','Familiarity Reaction Time (s)','Memory Image']]

,Familiarity Rating,Familiarity Reaction Time (s),Memory Image
10,1.0,0.520,00952960627fa.jpg
11,NaN,0.000,sunapqwoeotxshkukbh.jpg
12,4.0,1.020,sunafvwfsmqkjrutjta.jpg
13,2.0,0.753,sunabllobxadqcyttej.jpg
14,2.0,0.519,sunacepzfrmtkiqonpy.jpg
15,1.0,0.787,sunaiycurtqsfkvcaso.jpg
16,3.0,0.353,00543940519fa.jpg
17,3.0,0.204,00775941205fa.jpg
18,NaN,0.000,00779941205fa.jpg
19,2.0,0.252,00926960627Fa.jpg


# Check attention probe RT's

In [39]:
log_file[((log_file['DATA'].str.contains('ATTN PROBE')) & (log_file['DATA'].str.contains('ON'))) | (log_file['DATA'].str.contains('Keypress'))].iloc[10:30][['DATA','TIME']]

,DATA,TIME
199,29.5553 \tDATA \tKeypress: 3,29.5553
214,33.2049 \tEXP \tWIN FLIP : STIMULUS ATTN PROBE...,33.2049
215,34.0742 \tDATA \tKeypress: 1,34.0742
241,40.8414 \tEXP \tWIN FLIP : STIMULUS ATTN PROBE...,40.8414
242,41.7257 \tDATA \tKeypress: 3,41.7257
268,48.3765 \tEXP \tWIN FLIP : STIMULUS ATTN PROBE...,48.3765
269,48.9789 \tDATA \tKeypress: 3,48.9789
275,50.9143 \tDATA \tKeypress: space,50.9143
397,53.2302 \tDATA \tKeypress: 3,53.2302
584,55.7658 \tDATA \tKeypress: 2,55.7658


In [40]:
a = log_file[(log_file['DATA'].str.contains('ATTENTION PROBE ON'))]['DATA'].str.split('\t', 4).str

In [41]:
probe_indices = list(log_file[(log_file['DATA'].str.contains('ATTN PROBE ON'))].index)

because these lines are long, they got split into two lines when converting the log file into csv format.
if we want to look at the psychopy clock times, we have to retrieve them from the line before the attention probe line. so, we get the indices of the lines where the second half of the attention probe info displays, and subtract each by 1

In [42]:
probe_time_indices = [x-1 for x in probe_indices]
key_press_indices = [x+1 for x in probe_indices]

In [43]:
for idx,x in enumerate(key_press_indices):
    while 'Keypress: 1' not in log_file.loc[x]['DATA'] and 'Keypress: 3' not in log_file.loc[x]['DATA']:
        x+=1
    key_press_indices[idx]=x

In [44]:
probe_time_indices

[213, 240, 267, 1329, 1360, 1392, 1423, 1454, 1485, 1516, 1547, 1578, 1609]

In [45]:
full_index = probe_time_indices+key_press_indices+probe_indices
full_index.sort()

In [46]:
attn_rt = {}
attn_rt['probe_start'] = list(log_file.loc[probe_time_indices]['TIME'][3:])
attn_rt['key press'] = list(log_file.loc[key_press_indices]['TIME'][3:])
attn_df = pd.DataFrame(attn_rt)
attn_df

,probe_start,key press
0,70.0841,70.6851
1,77.4692,78.6044
2,85.3717,86.2068
3,92.9741,93.5591
4,100.3274,101.0784
5,107.8458,108.8317
6,115.5991,116.6826
7,123.4508,124.3698
8,131.1203,131.5883
9,138.3554,138.9910


In [47]:
log_file_rt = attn_df['key press'].astype('float64')-attn_df['probe_start'].astype('float64')
csv_rt = exp_raw[exp_raw['Trial Type']=='Presentation']['Attention Reaction Time (s)']
thisdict = {'log rt':log_file_rt, 'csv rt':csv_rt}
df = pd.DataFrame(thisdict)

In [48]:
df

,log rt,csv rt
0,0.6010,0.600915
1,1.1352,1.135215
2,0.8351,0.835113
3,0.5850,0.585055
4,0.7510,0.750887
5,0.9859,0.985893
6,1.0835,1.083401
7,0.9190,0.919017
8,0.4680,0.467985
9,0.6356,0.635558


In [49]:
df['diff'] = df['csv rt'] - df['log rt']

In [50]:
print('Mean difference in response times between csv and log file: '+str(df['diff'].mean()))

Mean difference in response times between csv and log file: -2.613037109426841e-05


### Responses from log file

In [51]:
list(log_file.loc[key_press_indices]['DATA'][3:])

['70.6851 \tDATA \tKeypress: 1',
 '78.6044 \tDATA \tKeypress: 3',
 '86.2068 \tDATA \tKeypress: 1',
 '93.5591 \tDATA \tKeypress: 1',
 '101.0784 \tDATA \tKeypress: 1',
 '108.8317 \tDATA \tKeypress: 1',
 '116.6826 \tDATA \tKeypress: 1',
 '124.3698 \tDATA \tKeypress: 3',
 '131.5883 \tDATA \tKeypress: 1',
 '138.9910 \tDATA \tKeypress: 3']

### Responses from csv

In [52]:
exp_raw[['Attention Button']].iloc[0:10]

,Attention Button
0,1.0
1,3.0
2,1.0
3,1.0
4,1.0
5,1.0
6,1.0
7,3.0
8,1.0
9,3.0
